In [19]:
# from selenium.webdriver.support.ui import Select

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from PIL import Image
import pytesseract

caminho = r"C:\Users\hconstancio\AppData\Local\Tesseract-OCR"
pytesseract.pytesseract.tesseract_cmd = caminho + r'\tesseract.exe'

def getTextoCaptcha():
    time.sleep(5)

    # Localiza o Captcha no site
    captcha = navegador.find_element(By.XPATH, '//*[@id="img"]')
    captcha.screenshot("captcha.png") # Salva o Captcha como imagem

    # Processa a imagem do CAPTCHA
    imagem = Image.open("captcha.png")
    texto_captcha = pytesseract.image_to_string(imagem)

    return texto_captcha

time.sleep(5)

navegador = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

navegador.implicitly_wait(20) # seconds
navegador.get('https://www2.rio.rj.gov.br/smf/iptulogradouro/')


navegador.find_element(By.XPATH, '/html/body/div[1]/div/div/form/table/tbody/tr[2]/td[1]/input').send_keys('URUCUIA')

time.sleep(5)
captcha = getTextoCaptcha()

In [ ]:
try:
    pass
except:
    codigo_errado = navegador.find_element(By.XPATH, '/html/body/div[1]/div/center/div/table[1]/tbody/tr[3]/td/font[1]/b')
    navegador.find_element(By.XPATH, '/html/body/div[1]/div/center/div/table[2]/tbody/tr/td/div/form/input').click()

In [ ]:
import pandas as pd

# Indicando onde está o arquivo
planilha = 'iptu'
codigos = pd.read_excel(f'{planilha}.xlsx')

# Removendo linhas duplicadas com base em uma coluna
# codigos = codigos.drop_duplicates(subset=['codigo'])

# Excluindo valores nan
codigos = codigos.dropna(subset=['cl'])
# Convertendo float em int e salvando uma lista com tolist()
codigos = codigos['cl'].tolist()

# codigos =[359931, 359963, 359743]
len(codigos)

1818

In [32]:
codigos =[3400, 4317, 13441]

In [ ]:
def dadosCodigo():
    # Capturar o valor pesquisado e o exercício
    pesquisado = navegador.find_element(By.XPATH, "//tr[2]/td[1]").text.strip()
    exercicio = navegador.find_element(By.XPATH, "//tr[2]/td[2]").text.strip()

    # Capturar todas as linhas de logradouros
    linhas = navegador.find_elements(By.XPATH, "//tr[@class='linha']")

    # Preparar lista para armazenar os dados
    dados = []

    for linha in linhas:
        try:
            codigo_logradouro = linha.find_element(By.XPATH, "./td[2]").text.strip()
            logradouro = linha.find_element(By.XPATH, "./td[3]").text.strip()
            bairro = linha.find_element(By.XPATH, "./td[4]").text.strip()
            dados.append({
                "Pesquisado": pesquisado,
                "Exercício": exercicio,
                "CódigoLogradouro": codigo_logradouro,
                "Logradouro": logradouro,
                "Bairro": bairro
            })
        except Exception as e:
            print("Erro ao processar linha:", str(e))

    # Criar um DataFrame com os dados
    df = pd.DataFrame(dados)

    # Exportar para Excel
    df.to_excel("tabela_formatada.xlsx", index=False)

    navegador.get('https://www2.rio.rj.gov.br/smf/iptulogradouro/')

In [ ]:
for codigo in codigos:
    navegador.find_element(By.XPATH, '/html/body/div[1]/div/div/form/table/tbody/tr[3]/td[1]/input').send_keys(codigo)

    try:
        captcha = getTextoCaptcha()
        navegador.find_element(By.XPATH, '//*[@id="texto_imagem"]').send_keys(captcha)
        
        navegador.find_element(By.XPATH, '/html/body/div[1]/div/div/form/table/tbody/tr[5]/td/input').click()

        dadosCodigo()
    except:

        while alert != '':
            alert = navegador.switch_to.alert
            alert.accept()

            captcha = getTextoCaptcha()
            navegador.find_element(By.XPATH, '//*[@id="texto_imagem"]').send_keys(captcha)
            
            navegador.find_element(By.XPATH, '/html/body/div[1]/div/div/form/table/tbody/tr[5]/td/input').click()

            dadosCodigo()
    
    